In [52]:
!pip install evaluate
!pip install seqeval
!pip install wandb --upgrade

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:

import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorForTokenClassification
import pandas as pd
from sklearn.model_selection import train_test_split
import ast
from tqdm import tqdm
import numpy as np
import torch


In [54]:
df = pd.read_csv("/kaggle/input/named-entity-recognition-ner-corpus/ner.csv")

In [55]:
import ast

def length_check(x):
    if len(x.Sentence)==len(x.Tag):
        return 1
    else:
        return 0

    
def preprocess_data(data):
    for i in range(len(data)):
        pos = ast.literal_eval(data['POS'][i])
        tags = ast.literal_eval(data['Tag'][i])
        data['POS'][i] = [str(word) for word in pos]
        data['Tag'][i] = [str(word.upper()) for word in tags]
    return data

data = preprocess_data(df)

data["Sentence"]=data.Sentence.apply(lambda x: x.split(" "))

data["length"]=data.apply(lambda x: length_check(x),axis=1)
print(data["length"].value_counts())
df = data[["Sentence","Tag"]]


/tmp/ipykernel_36/1546122762.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['POS'][i] = [str(word) for word in pos]
/tmp/ipykernel_36/1546122762.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
Yo

length
1    47959
Name: count, dtype: int64


In [56]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [57]:
df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)

In [58]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [59]:
df_train

,Sentence,Tag
7707,"[The, 58-year-old, former, analyst, says, he, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GPE,..."
26089,"[But, he, said, he, will, not, accept, any, Pa...","[O, O, O, O, O, O, O, O, B-GPE, O, O, O, O, O,..."
308,"[The, Swiss, star, was, upset, Wednesday, by, ...","[O, B-GPE, O, O, O, B-TIM, O, B-GPE, B-PER, I-..."
24975,"[After, taking, office, in, 2004, ,, the, SPEN...","[O, O, O, O, B-TIM, O, O, B-ORG, O, O, O, O, O..."
471,"[Israeli, soldiers, have, killed, two, Palesti...","[B-GPE, O, O, O, O, B-GPE, O, O, O, B-GEO, I-G..."
...,...,...
11284,"[However, ,, poverty, ,, illiteracy, ,, and, u...","[O, O, O, O, O, O, O, O, O, O, O, O]"
44732,"[The, report, was, published, two, days, after...","[O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O..."
38158,"[In, a, separate, incident, ,, kidnappers, rel...","[O, O, O, O, O, O, O, O, O, O, O, B-TIM, O, O,..."
860,"[Those, groups, were, shut, down, by, U.S., of...","[O, O, O, O, O, O, B-ORG, O, O, O, B-TIM, O]"


In [60]:

flattened_set_train = set(value for sublist in df_train.Tag.to_list() for value in sublist)
print(len(flattened_set_train))

print(flattened_set_train)

print("****"*20)
flattened_set_test = set(value for sublist in df_test.Tag.to_list() for value in sublist)
print(len(flattened_set_test))
print(flattened_set_test)

labels_to_ids = {k: v for v, k in enumerate(flattened_set_train)}
ids_to_labels = {v: k for v, k in enumerate(flattened_set_test)}
print(labels_to_ids)
print(ids_to_labels)

17
{'I-EVE', 'B-ART', 'B-GPE', 'I-ORG', 'I-NAT', 'O', 'I-GEO', 'I-ART', 'I-PER', 'B-TIM', 'B-NAT', 'I-GPE', 'B-GEO', 'B-EVE', 'B-PER', 'I-TIM', 'B-ORG'}
********************************************************************************
17
{'I-EVE', 'B-ART', 'B-GPE', 'I-ORG', 'I-NAT', 'O', 'I-GEO', 'I-ART', 'I-PER', 'B-TIM', 'I-GPE', 'B-NAT', 'B-GEO', 'B-EVE', 'B-PER', 'I-TIM', 'B-ORG'}
{'I-EVE': 0, 'B-ART': 1, 'B-GPE': 2, 'I-ORG': 3, 'I-NAT': 4, 'O': 5, 'I-GEO': 6, 'I-ART': 7, 'I-PER': 8, 'B-TIM': 9, 'B-NAT': 10, 'I-GPE': 11, 'B-GEO': 12, 'B-EVE': 13, 'B-PER': 14, 'I-TIM': 15, 'B-ORG': 16}
{0: 'I-EVE', 1: 'B-ART', 2: 'B-GPE', 3: 'I-ORG', 4: 'I-NAT', 5: 'O', 6: 'I-GEO', 7: 'I-ART', 8: 'I-PER', 9: 'B-TIM', 10: 'I-GPE', 11: 'B-NAT', 12: 'B-GEO', 13: 'B-EVE', 14: 'B-PER', 15: 'I-TIM', 16: 'B-ORG'}


In [61]:
labels_to_ids = {'B-ORG': 0, 'I-GPE': 1, 'I-GEO': 2, 'I-ORG': 3, 'B-PER': 4, 'I-PER': 5, 'I-ART': 6, 'B-GEO': 7, 'B-EVE': 8, 'I-EVE': 9, 'I-NAT': 10, 'B-ART': 11, 'I-TIM': 12, 'O': 13, 'B-TIM': 14, 'B-NAT': 15, 'B-GPE': 16}
ids_to_labels = {0: 'B-ORG', 1: 'I-GPE', 2: 'I-GEO', 3: 'I-ORG', 4: 'B-PER', 5: 'I-PER', 6: 'I-ART', 7: 'I-NAT', 8: 'B-EVE', 9: 'I-EVE', 10: 'B-GPE', 11: 'B-ART', 12: 'I-TIM', 13: 'O', 14: 'B-TIM', 15: 'B-NAT', 16: 'B-GEO'}

In [62]:
df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

In [63]:
# df_train.iloc[0]["Sentence"]
df_train.iloc[0]["Tag"]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GEO',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O']

In [64]:
def tokenize_and_align_labels(df,tokenizer):
    tokenized_inputs = tokenizer(df["Sentence"].to_list(),padding=True,max_length=128,is_split_into_words=True)
    labels = []
    for i, label in enumerate(df[f"Tag"].to_list()):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append([labels_to_ids[i] if i!= -100 else -100 for i in label_ids])

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [65]:
tokenized_dataset_train = tokenize_and_align_labels(df_train,tokenizer)
tokenized_dataset_val = tokenize_and_align_labels(df_test,tokenizer)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [66]:
tokenized_dataset_train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [67]:
# convert the tokenized data into the format [[dictinoary containing tokenized results]]
tokenized_dataset_train_updated = [{key: tokenized_dataset_train[key][i] for key in tokenized_dataset_train.keys()} for i in range(len(tokenized_dataset_train['input_ids']))]

tokenized_dataset_val_updated = [{key: tokenized_dataset_val[key][i] for key in tokenized_dataset_val.keys()} for i in range(len(tokenized_dataset_val['input_ids']))]


In [68]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=17, id2label=ids_to_labels, label2id=labels_to_ids)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [69]:
training_args = TrainingArguments(
    output_dir="ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [70]:
data_collator=DataCollatorForTokenClassification(tokenizer)

In [71]:
seqeval = evaluate.load("seqeval")

In [72]:
# Other parts of your code remain the same...

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ids_to_labels[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ids_to_labels[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [73]:
tokenized_dataset_train_updated[1]

{'input_ids': [101,
  2021,
  2002,
  2056,
  2002,
  2097,
  2025,
  5138,
  2151,
  9889,
  6378,
  2008,
  7336,
  2417,
  2527,
  9328,
  1996,
  2240,
  1997,
  2491,
  2008,
  18600,
  2796,
  1011,
  2013,
  9889,
  1011,
  4758,
  13329,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [74]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train_updated,
    eval_dataset=tokenized_dataset_val_updated,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.117600,0.112753,0.807524,0.815208,0.811348,0.966381


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2398, training_loss=0.15567971071270328, metrics={'train_runtime': 677.4231, 'train_samples_per_second': 56.637, 'train_steps_per_second': 3.54, 'total_flos': 3015792894382884.0, 'train_loss': 0.15567971071270328, 'epoch': 1.0})

In [77]:
import torch

def infer_entities(sentence, tokenizer, model, labels_to_ids, ids_to_labels, max_len=128):
    """
    Infer entities for a given sentence using the trained model.

    Args:
        sentence (str): The input sentence for which to infer entities.
        tokenizer (AutoTokenizer): The tokenizer used for encoding the input.
        model (AutoModelForTokenClassification): The trained model for token classification.
        labels_to_ids (dict): Mapping of labels to IDs.
        ids_to_labels (dict): Mapping of IDs to labels.
        max_len (int): Maximum sequence length for tokenization.

    Returns:
        list: A list of (token, predicted_label) tuples.
    """
    # Move the model to the same device as the input tensors
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input sentence
    tokenized_input = tokenizer(sentence.split(), padding=True, truncation=True, max_length=max_len, is_split_into_words=True, return_tensors="pt")

    # Move input tensors to the same device as the model
    tokenized_input = {key: value.to(device) for key, value in tokenized_input.items()}

    # Ensure model is in evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        outputs = model(**tokenized_input)

    # Get predictions
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2).squeeze().tolist()

    # Map token IDs to label names
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"].squeeze().tolist())
    predicted_labels = [ids_to_labels[pred] for pred in predictions]

    # Remove special tokens
    tokens = [token for token in tokens if token not in tokenizer.all_special_tokens]
    predicted_labels = [label for label, token in zip(predicted_labels, tokens) if token not in tokenizer.all_special_tokens]

    # Create a list of (token, predicted_label) tuples
    entity_results = list(zip(tokens, predicted_labels))

    return entity_results

# Example usage
sentence = "The International Atomic Energy Agency is to hold second day of talks in Vienna Wednesday on how to respond to Iran's resumption of low-level uranium conversion."
entities = infer_entities(sentence, tokenizer, model, labels_to_ids, ids_to_labels)
print(entities)


[('the', 'O'), ('international', 'O'), ('atomic', 'B-ORG'), ('energy', 'I-ORG'), ('agency', 'I-ORG'), ('is', 'I-ORG'), ('to', 'O'), ('hold', 'O'), ('second', 'O'), ('day', 'O'), ('of', 'O'), ('talks', 'O'), ('in', 'O'), ('vienna', 'O'), ('wednesday', 'I-NAT'), ('on', 'B-TIM'), ('how', 'O'), ('to', 'O'), ('respond', 'O'), ('to', 'O'), ('iran', 'O'), ("'", 'I-NAT'), ('s', 'O'), ('res', 'O'), ('##ump', 'O'), ('##tion', 'O'), ('of', 'O'), ('low', 'O'), ('-', 'O'), ('level', 'O'), ('uranium', 'O'), ('conversion', 'O'), ('.', 'O')]
